## Задание, оцениваемое сокурсниками: Соревнование по сентимент-анализу

В этом задании вам нужно воспользоваться опытом предыдущих недель, чтобы побить бейзлайн в соревновании по сентимент-анализу отзывов на товары на Kaggle Inclass:

https://inclass.kaggle.com/c/product-reviews-sentiment-analysis-light

В качестве ответа в этом задании вам нужно загрузить ноутбук с решением и скриншот вашего результата на leaderboard.

Убедитесь, что:

1) ход вашего решения задокументирован достаточно подробно для того, чтобы ваши сокурсники поняли, что вы делали и почему,

2) ваша команда в соревновании состоит только из вас и названа вашим логином на Сoursera, чтобы ваши сокурсники могли понять, что на скриншоте именно ваш результат



### Сентимент-анализ отзывов на товары (простая версия)
Классифицируйте отзывы по тональности

Description: В этом соревновании вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная. В отличие от усложненной версии задачи, здесь вам не требуется самостоятельно собирать обучающую выборку - она есть в предоставляемых вам данных.

Evaluation: В качестве метрики качества используется accuracy. При валидации качества не забывайте, что ваш результат точно должен быть лучше, чем тривиальные ответы (всегда 0, всегда 1, случайный выбор класса).

необходимые файлы

products_sentiment_train.tsv - тренировочный сет

products_sentiment_test.tsv - тестовый сет

products_sentiment_sample_submission.csv - опример ответов тестового сета

In [1]:
#формируем тренировочный датасет из файла
import pandas as pd
import sklearn
print(sklearn.__version__)

train_data =  pd.DataFrame.from_csv('products_sentiment_train.tsv', sep='\t', header = None, index_col=False)
train_data.columns = ['review', 'class']
print(train_data.shape)
train_data.head()
print(sum(train_data['class'] == 1))
print(sum(train_data['class'] != 1))
#формируем два списка: отзывы в текстовом формате и метки классов в виде чисел 1 или 0
reviews_train = list(train_data['review'])
class_train = list(train_data['class'])

0.18.1
(2000, 2)
1274
726


в тренировочном датасете мы имеем 2000 отзывов из них 1274 положительных и 726 отрицательных

In [2]:
#формируем тестовый датасет из файла
test_data =  pd.read_csv('products_sentiment_test.tsv', sep = "\t+", index_col = 'Id', engine='python')
print(test_data.shape)
test_data.head()
#формируем список: отзывы в текстовом формате
reviews_test = list(test_data['text'])

(500, 1)


в тестовом датасете мы имеем 500 отзывов, к какому классу они принадлежат, нам предлагается определить

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

загружаем необходимые библиотеки

In [4]:
#пробовал выбирать вручную
#CountVectorizer и LogisticRegression с параметрами по умолчанию
text_clf1 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
print(round(cross_val_score(text_clf1, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))


#CountVectorizer и метод опорных векторов
#previous score of 0.78500
text_clf2 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5))])
print(round(cross_val_score(text_clf2, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

#CountVectorizer и метод Логистическая регрессия
text_clf3 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression())])
print(round(cross_val_score(text_clf3, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

#TfidfVectorizer и метод Логистическая регрессия
text_clf4 = Pipeline([('vect', TfidfVectorizer()),
                       ('clf', LogisticRegression())])
print(round(cross_val_score(text_clf4, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

#TfidfVectorizer и метод LinearSVC
text_clf5 = Pipeline([('vect', CountVectorizer()),
                       ('clf', LinearSVC())])
print(round(cross_val_score(text_clf5, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

#CountVectorizer и метод опорных векторов + биграммы
text_clf6 = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     #('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5))])
print(round(cross_val_score(text_clf6, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

#CountVectorizer и метод опорных векторов + н-граммы
text_clf7 = Pipeline([('vect', CountVectorizer(ngram_range=(3, 5), analyzer='char_wb')),
                     #('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5))])
print(round(cross_val_score(text_clf7, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

#CountVectorizer и метод опорных векторов + н-граммы 
#Your submission scored 0.79250, which is an improvement of your previous score of 0.78500. Great job!
text_clf8 = Pipeline([('vect', CountVectorizer(stop_words = 'english', ngram_range=(3, 5), analyzer='char_wb')),
                     #('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5))])
print(round(cross_val_score(text_clf8, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

#CountVectorizer и метод опорных векторов + н-граммы 
text_clf9 = Pipeline([('vect', CountVectorizer(stop_words = 'english', ngram_range=(2, 4), analyzer='char_wb')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=50))])
print(round(cross_val_score(text_clf9, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

0.707
0.766
0.7665
0.7665
0.754
0.759
0.754
0.7405
0.794


### Your submission scored 0.79250, which is not an improvement of your best score. Keep trying!
с параметрами по умолчанию, или с настройками, подсмотренными из классификаторов других примеров, мы не можем достичь требуемого качества, поэтому на следующих шагах будем использовать поиск по сетке

In [5]:
#сетка для векторайзера tfidf  и LogisticRegression
%time
from sklearn.model_selection import GridSearchCV

def tokenizer(text):
    return text.split()

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

from nltk.corpus import stopwords
stop = stopwords.words('english')

tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase = False,
                       preprocessor = None)
param_grid = [{'vect__ngram_range': [(1,1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm': [None],               
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf',
                    LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                          scoring = 'accuracy',
                          cv = 5, verbose = 1,
                          n_jobs = 1)
gs_lr_tfidf.fit(reviews_train, class_train)

clf = gs_lr_tfidf.best_estimator_
print(round(cross_val_score(clf, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

Wall time: 0 ns
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:  3.1min finished


0.7825


качество работы (0,7825 - для лучшей связки векторизатор-классификатор из сетки) нас не устравает. Но уже на этом этапе мы достигли бейзлайна конкурса. Пробуем создать другую сетку (другие векторизаторы и классификаторы)

### Your submission scored 0.80000
[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=False, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenizer_porter at 0x000000000A345510>,
          use_idf=True, vocabulary=None)),
 ('clf',
  LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [6]:
countvect = CountVectorizer(strip_accents = None,
                       lowercase = False,
                       preprocessor = None)
param_grid = [{'vect__ngram_range': [(1,1),(1,2),(2,2),(1,3),(2,3)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter, None],
               'clf__loss': ['hinge', 'log', 'modified_huber', 'squared_hinge'],
               'clf__penalty': ['l1', 'l2'],
               'clf__alpha': [1e-3, 1e-4],
               'clf__n_iter': [50,100,200]},
              {'vect__ngram_range': [(1,1),(1,2),(2,2),(1,3),(2,3)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer, tokenizer_porter, None],
              'clf__loss': ['hinge', 'log', 'modified_huber', 'squared_hinge'],
              'clf__penalty': ['l1', 'l2', 'elasticnet'],
              'clf__alpha': [1e-3, 1e-4],
              'clf__n_iter': [50,100,200]},
              ]
sgd_countv = Pipeline([('vect', countvect),
                    ('clf',
                    SGDClassifier(random_state=0))])
gs_sgd_countv = GridSearchCV(sgd_countv, param_grid,
                          scoring = 'accuracy',
                          cv = 5, verbose = 1,
                          n_jobs = 1)
gs_sgd_countv.fit(reviews_train, class_train)

clf = gs_sgd_countv.best_estimator_
print(round(cross_val_score(clf, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

Fitting 5 folds for each of 3600 candidates, totalling 18000 fits


[Parallel(n_jobs=1)]: Done 18000 out of 18000 | elapsed: 211.4min finished


0.7965


поиск по сетке CountVectorizer+SGDClassifier позволил достичь результата 0,81250

### Your submission scored 0.81250
[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=False, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 2), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenizer_porter at 0x000000000A345510>,
          vocabulary=None)),
 ('clf',
  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
         eta0=0.0, fit_intercept=True, l1_ratio=0.15,
         learning_rate='optimal', loss='hinge', n_iter=200, n_jobs=1,
         penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
         warm_start=False))]

In [7]:
#лучший классификатор для сетки CountVectorizer+SGDClassifier
text_clf_best_gs_sgd = Pipeline([('vect', CountVectorizer(stop_words = None, ngram_range=(1, 2),
                                              analyzer = 'word', binary = False,
                                              decode_error = 'strict', 
                                              #dtype= <class 'numpy.int64'>, 
                                              encoding='utf-8', input='content',
                                              lowercase=False, max_df=1.0, max_features=None, min_df=1,
                                              preprocessor=None, strip_accents=None, 
                                              token_pattern='(?u)\\b\\w\\w+\\b',
                                              tokenizer=tokenizer_porter,
                                              vocabulary=None)),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           average = False, class_weight = None,
                                           epsilon = 0.1, eta0 = 0.0, 
                                           fit_intercept = True, l1_ratio = 0.15,
                                           learning_rate = 'optimal',
                                           shuffle = True, verbose = 0,
                                           warm_start = False,
                                           alpha=1e-3, n_iter=200))])
print(round(cross_val_score(text_clf_best_gs_sgd, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

0.785


In [8]:
text_clf_best_gs_sgd1 = Pipeline([('vect', CountVectorizer(stop_words = None, ngram_range=(1, 2),
                                              analyzer = 'word', binary = False,
                                              decode_error = 'strict', 
                                              #dtype= <class 'numpy.int64'>, 
                                              encoding='utf-8', input='content',
                                              lowercase=False, max_df=1.0, max_features=None, min_df=1,
                                              preprocessor=None, strip_accents=None, 
                                              token_pattern='(?u)\\b\\w\\w+\\b',
                                              tokenizer=tokenizer_porter,
                                              vocabulary=None)),
                     ('clf', SGDClassifier(loss= 'hinge', penalty='l2',
                                           average = True, class_weight = None,
                                           epsilon = 0.1, eta0 = 0.0, 
                                           fit_intercept = True, l1_ratio = 0.15,
                                           learning_rate = 'optimal',
                                           shuffle = True, verbose = 0,
                                           warm_start = False,
                                           alpha=1e-3, n_iter=200))])
print(round(cross_val_score(text_clf_best_gs_sgd1, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

0.793


на этом шаге я попробовал вручную поменять некоторые параметры, и мне удалось улучшить качество, изменив параметр average = True классификатора
### Your submission scored 0.81500, which is an improvement of your previous score of 0.81250. Great job!

попробуем создать сетку TfidfVectorizer+SGDCClassifier

In [9]:
tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase = False,
                       preprocessor = None)
param_grid = [{'vect__ngram_range': [(1,1), (1,2)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__loss': ['hinge', 'log', 'modified_huber', 'squared_hinge']},
              ]
sgd_tfidf = Pipeline([('vect', tfidf),
                    ('clf',
                    SGDClassifier(random_state=0))])
gs_sgd_tfidf = GridSearchCV(sgd_tfidf, param_grid,
                          scoring = 'accuracy',
                          cv = 5, verbose = 1,
                          n_jobs = 1)
gs_sgd_tfidf.fit(reviews_train, class_train)
clf = gs_sgd_tfidf.best_estimator_
print(round(cross_val_score(clf, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:  4.6min finished


0.795


In [10]:
clf3 = gs_sgd_tfidf.best_estimator_
#clf2 = gs_sgd_countv.best_estimator_
clf3.steps


[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=False, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenizer_porter at 0x000000000A33F510>,
          use_idf=True, vocabulary=None)),
 ('clf',
  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
         eta0=0.0, fit_intercept=True, l1_ratio=0.15,
         learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
         penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
         warm_start=False))]

[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=False, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenizer_porter at 0x000000000A345510>,
          use_idf=True, vocabulary=None)),
 ('clf',
  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
         eta0=0.0, fit_intercept=True, l1_ratio=0.15,
         learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
         penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
         warm_start=False))]

### Your submission scored 0.81500, which is not an improvement of your best score. Keep trying!

In [29]:
text_clf_best_gs_sgd2 = Pipeline([('vect', TfidfVectorizer(stop_words = None, ngram_range=(1, 2),
                                              analyzer = 'word', binary = False,
                                              decode_error = 'strict', 
                                              #dtype= <class 'numpy.int64'>, 
                                              encoding='utf-8', input='content',
                                              lowercase=False, 
                                              norm = 'l2', smooth_idf=True,
                                              max_df=1.0, max_features=None, min_df=1,
                                              preprocessor=None, strip_accents=None, 
                                              sublinear_tf=False, use_idf=True,
                                              token_pattern='(?u)\\b\\w\\w+\\b',
                                              tokenizer=tokenizer_porter,
                                              vocabulary=None)),
                     ('clf', SGDClassifier(loss= 'hinge', penalty='l2',
                                           average = False, class_weight = None,
                                           epsilon = 0.1, eta0 = 0.0, 
                                           fit_intercept = True, l1_ratio = 0.15,
                                           learning_rate = 'optimal',
                                           shuffle = True, verbose = 0,
                                           warm_start = False,
                                           power_t=0.3,
                                           alpha=1e-4, n_iter=100))])
print(round(cross_val_score(text_clf_best_gs_sgd2, reviews_train, class_train, scoring="accuracy", cv = 5).mean(),4))

0.795


поиском по третьей сетке мы повторили наш лучший результат, на этом, наверное, пока и остановимся.

In [30]:
#мы выбрали лучший классификатор, на нем формируем прогноз для тестовой выборки
text_clf_best = text_clf_best_gs_sgd2
text_clf_best.fit(reviews_train, class_train)
predicted = text_clf_best.predict(reviews_test)

In [31]:
#на основе предсказанных значений классификатора формируем dataframe согласно примеру sample_submission
#записываем его в файл sample_submission.csv, который загрузим в kaggle
ans  = pd.DataFrame(predicted)
ans.columns = ['y']
ans.index.name = 'Id'
ans.to_csv('products_sentiment_submission.csv')
ans.head()

,y
Id,
0,1
1,0
2,1
3,1
4,1
